### Generate a Complete Database 

- date : 24/05/2024
- New features : generate a complete database using "OrdalieTech/Solon-embeddings-large-0.1" embedding model (config file have been improved)
- Expected improvement : improving the retrieval capabilities with a much strong embedding model.

In [ ]:
import sys
import os
import pandas as pd

### Building New Complete Dataset based on config files 

In [ ]:
from db_building import  build_database_from_csv
#db = build_database_from_csv('/home/onyxia/work/llm-open-data-insee/data_complete.csv')
#db.similarity_search("Quels sont les chiffres du chômages en 2023")

### Loading Dataset based on config files

In [ ]:
from db_building import reload_database_from_local_dir
db = reload_database_from_local_dir(persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db")

In [ ]:
#check if there are at least one encoded document in our vectorstore
print(len(db.get()["ids"])) 

In [ ]:
result = db.similarity_search("Quels résultats au BAC les étudiants de classes préparatoires ont ils généralement?", k = 5 )
print(result[0])

In [ ]:
from config import RAG_PROMPT_TEMPLATE, EMB_MODEL_NAME, MODEL_NAME
from model_building import build_llm_model
from chain_building.build_chain import (
    load_retriever,
    build_chain
    )

from langchain_core.prompts import PromptTemplate
import chainlit as cl

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def build_chain_test(retriever, prompt, llm):
    """
    Build a LLM chain based on Langchain package and INSEE data
    """
    # Create a Langchain LLM Chain
    chain = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | prompt
        | llm
        | StrOutputParser()
    )

    rag_chain_with_source = RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    ).assign(answer=chain)

    return rag_chain_with_source

In [ ]:
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)
print(prompt)

In [ ]:
retriever = db.as_retriever(search_type="mmr", search_kwargs={"score_threshold": 0.5, "k": 5})

#retriever = load_retriever(emb_model_name=EMB_MODEL_NAME,
                        #persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db")

In [ ]:
llm = build_llm_model(model_name=MODEL_NAME,
                        quantization_config=True,
                        config=True, 
                        token = os.environ["HF_TOKEN"])

In [ ]:
chain = build_chain_test(retriever, prompt, llm)

In [ ]:
chain

In [ ]:
question = "Quel est le but initial derrière la création du système de retraites français après la Seconde Guerre mondiale?" 
#question = "Quelle est la cause principale de l'augmentation de l'indice des prix à la consommation (IPC)?"
results = retriever.invoke(question)

for i, doc in enumerate(results):
    print(f"Doc {i} : {doc.metadata["source"]}")
    print(doc.page_content)

In [ ]:
for chunk in chain.stream(question):
    print(chunk)

In [ ]:
answer = chain.invoke(question) 

In [ ]:
print(answer["answer"])

### Adding a Reranker 

The goal of this part is to build a pipeline Langchain where we have added a reranker: a BM25, a ColBERT model, a french cross-encoder, a multilingual cross-encoder and several hyperparameters.  

Reranker model list : 
- multilingual cross encoder : BAAI/bge-reranker-large (multilingual),
- french cross encoder : antoinelouis/crossencoder-electra-base-french-mmarcoFR  OR dangvantuan/CrossEncoder-camembert-large
- BM25 : langchain_community.retrievers import BM25Retriever
- ColBERT : antoinelouis/colbertv2-camembert-L4-mmarcoFR


In [ ]:
!mc cp s3/projet-llm-insee-open-data/data/chroma_database/chroma_db /home/onyxia/work/llm-open-data-insee/data --recursive

In [ ]:
sys.path.append("/home/onyxia/work/llm-open-data-insee/src") 

In [ ]:
from chain_building import (
    load_retriever
    )
from config import EMB_MODEL_NAME, MODEL_NAME

retriever = load_retriever(
                emb_model_name = EMB_MODEL_NAME,
                persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db",
                device="cuda",
                collection_name="insee_data"
            )


In [ ]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
#test embedding retriever 
question = "Comment est calculé le pouvoir d'achat ?" 
#question = "Quelle est la cause principale de l'augmentation de l'indice des prix à la consommation (IPC)?"
results = retriever.invoke(question)
pretty_print_docs(results) #OK

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder #CrossEncoder 
from ragatouille import RAGPretrainedModel #ColBERT
from langchain_community.retrievers import BM25Retriever #BM25

colBERT = RAGPretrainedModel.from_pretrained("antoinelouis/colbertv2-camembert-L4-mmarcoFR")
colBERT_retriever  = ContextualCompressionRetriever(base_compressor=colBERT.as_langchain_document_compressor(k=5), base_retriever=retriever)

compressed_docs = colBERT_retriever.invoke(question)
pretty_print_docs(compressed_docs)

In [ ]:
model = HuggingFaceCrossEncoder(model_name="dangvantuan/CrossEncoder-camembert-large") #"antoinelouis/crossencoder-electra-base-french-mmarcoFR")
compressor_1 = CrossEncoderReranker(model=model, top_n=5)
compression_retriever_1 = ContextualCompressionRetriever(
    base_compressor=compressor_1, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

In [ ]:
model = HuggingFaceCrossEncoder(model_name= "BAAI/bge-reranker-large")
compressor_2 = CrossEncoderReranker(model=model, top_n=5)
compression_retriever_2 = ContextualCompressionRetriever(
    base_compressor=compressor_2, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

In [ ]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers = [compression_retriever_1, compression_retriever_2, colBERT_retriever], weigths = [1/3,1/3,1/3])

compressed_docs = ensemble_retriever.invoke(question)
pretty_print_docs(compressed_docs)

In [ ]:
from typing import Any, List, Optional, Sequence, Dict
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor
from langchain.schema import Document
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnableParallel , RunnablePassthrough
from langchain_community.retrievers import BM25Retriever

from langchain_core.callbacks import Callbacks
import math
from collections import Counter
from langchain_core.output_parsers import StrOutputParser

# Define the compression function
def compress_documents_lambda(documents: Sequence[Document], query: str, k: int = 5, **kwargs: Dict[str, Any]) -> Sequence[Document]:
    """Compress retrieved documents given the query context."""

    # Initialize the retriever with the documents
    retriever = BM25Retriever.from_documents(documents, k=k, **kwargs)
    relevant_docs = retriever.get_relevant_documents(query)
    return relevant_docs

# Define the complete chain
bm25_retriever = (
    RunnableParallel(
    {"documents": retriever, "query": RunnablePassthrough()}
    ) 
    | RunnableLambda(lambda r : compress_documents_lambda(documents= r["documents"] , query = r["query"]))
)

bm25_retriever.invoke(question)

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder #CrossEncoder 

model = HuggingFaceCrossEncoder(model_name="dangvantuan/CrossEncoder-camembert-large") #"antoinelouis/crossencoder-electra-base-french-mmarcoFR")
compressor_1 = CrossEncoderReranker(model=model, top_n=5)

compression_retriever_cross_encoder = ContextualCompressionRetriever(
    base_compressor=compressor_1, base_retriever=retriever
)

emsemble_reranking = EnsembleRetriever(retrievers = [compression_retriever_cross_encoder, bm25_retriever], weigths = [0.5, 0.5])


In [ ]:
emsemble_reranking.invoke(question)

### Adding a LLM Reranker

In [ ]:
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig,
    TextStreamer, 
    pipeline,
    TextStreamer
)
import torch
import torch.nn.functional as F

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# quantization config 
quantization_config  = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype="float16",
            bnb_4bit_use_double_quant=False,
        )

config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config,
    config=config
)

#### Fine-grained label reranker 

In [ ]:
# reranking utils function 

def expected_relevance_values(logits, grades_token_ids, list_grades):
    next_token_logits = logits[:, -1, :]
    next_token_logits = next_token_logits.cpu()[0]
    probabilities = F.softmax(next_token_logits[grades_token_ids], dim=-1).numpy()
    return np.dot(np.array(list_grades), probabilities)

def peak_relevance_likelihood(logits, grades_token_ids, list_grades):
    index_max_grade = np.array(list_grades).argmax()
    next_token_logits = logits[:, -1, :]
    probabilities = F.softmax(next_token_logits, dim=-1).cpu().numpy()[0]
    return probabilities[grades_token_ids[index_max_grade]]

def find_sublist_indices(main_list, sublist):
    sublist_length = len(sublist)
    main_list_length = len(main_list)
    
    # Helper function to check if sublist matches
    def is_sublist_at_index(index):
        return main_list[index:index + sublist_length] == sublist
    
    # Finding the first index
    first_index = -1
    
    for i in range(main_list_length - sublist_length + 1):
        if is_sublist_at_index(i):
            first_index = i
            break

    return first_index

In [ ]:
## assessing methods 
def RG_S(tokenizer, model,query, document, aggregating_method, k=5):

    list_grades = list(range(k))
    grades_token_ids = [tokenizer(str(grade))["input_ids"][1] for grade in list_grades]
 
    RG_S_template = """
    Sur une échelle de 0 à {k}, jugez la pertinence entre la requête et le document.
    Requête : {query}
    Document : {document}
    Réponse : """ 

    messages = [
        {"role": "system", "content": "Tu es un assistant chatbot expert en Statistique Publique."},
        {"role": "user", "content": RG_S_template.format(query=query, document=document, k=k)},
    ]

    input_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )

    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return aggregating_method(logits, grades_token_ids, list_grades)


def RG_4L(tokenizer, model,query, document, args):
    possible_judgements = [" Parfaitement Pertinent", " Très Pertinent", " Assez Pertinent", " Non Pertinent"]
    list_grades = np.array([3, 2, 1, 0])
    RG_4L_template = """
    Evaluez la pertinence du document donné par rapport à la question posée.
    Répondez uniquement parmi : Parfaitement Pertinent, Très Pertinent, Assez Pertinent ou Non Pertinent.
    Requête : {query}
    Document : {document}
    Réponse : {judgement}"""

    messages = [
        {"role": "system", "content": "Tu es un assistant chatbot expert en Statistique Publique."},
        {"role": "user", "content": RG_4L_template},
    ]

    log_probs = []
    for judgement in possible_judgements:
        input_text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=False,
            tokenize=False
        ).format(query=query, document=document, judgement=judgement)
        log_probs.append(compute_sequence_log_probs(sequence=input_text))

    probs = F.softmax(torch.tensor(log_probs), dim=-1).numpy()
    return np.dot(probs, list_grades)

def RG_3L(tokenizer, model,query, document, args):
    possible_judgements = [" Très Pertinent", " Assez Pertinent", " Non Pertinent"]
    list_grades = np.array([2, 1, 0])
    RG_3L_template = """
    Evaluez la pertinence du document donné par rapport à la question posée.
    Répondez uniquement parmi : Très Pertinent, Assez Pertinent ou Non Pertinent.
    Requête : {query}
    Document : {document}
    Réponse : {judgement}"""

    messages = [
        {"role": "system", "content": "Tu es un assistant chatbot expert en Statistique Publique."},
        {"role": "user", "content": RG_3L_template},
    ]

    log_probs = []
    for judgement in possible_judgements:
        input_text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=False,
            tokenize=False
        ).format(query=query, document=document, judgement=judgement)
        log_probs.append(compute_sequence_log_probs(sequence=input_text))

    probs = F.softmax(torch.tensor(log_probs), dim=-1).numpy()
    return np.dot(probs, list_grades)
    
def RG_YN(tokenizer, model,query, document, aggregating_method):
    list_judgements = [" Oui", " Non"]
    grades_token_ids = [tokenizer(j)["input_ids"][1] for j in list_judgements]
    list_grades = [1, 0]

    RG_YN_template = """
    Pour la requête et le document suivants, jugez s'ils sont pertinents. Répondez UNIQUEMENT par Oui ou Non.
    Requête : {query}
    Document : {document}
    Réponse : """

    messages = [
        {"role": "system", "content": "Tu es un assistant chatbot expert en Statistique Publique."},
        {"role": "user", "content": RG_YN_template.format(query=query, document=document)},
    ]

    input_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )

    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return aggregating_method(logits, grades_token_ids, list_grades)


In [ ]:
def llm_reranking(tokenizer, model,  query, retrieved_documents, assessing_method, aggregating_method):
    docs_content = retrieved_documents.copy() #[doc.page_content for doc in retrieved_documents]

    scores = []
    for document in docs_content:  
        score = assessing_method(tokenizer, model, query, document, aggregating_method)
        scores.append(score)

    docs_with_scores = list(zip(retrieved_documents, scores))
    docs_with_scores.sort(key=lambda x: x[1], reverse=True)
    sorted_documents = [doc for doc, score in docs_with_scores] #docs_with_scores 
    return sorted_documents

In [ ]:
expected_relevance_values(query="Combien y a t il d'habitant en France en 2024?", document="Le nombre d'habitant en Birmanie est de 54 millions de personnes")

In [ ]:
question = "Comment le taux de chômage en France a-t-il évolué au cours des dix dernières années ?"

documents = [
    "Le taux de chômage en France a connu des variations significatives au cours des dix dernières années. En 2014, le taux de chômage s'élevait à environ 10 %. Après un pic en 2015 à près de 10,5 %, il a progressivement diminué pour atteindre 8 % en 2019. La crise sanitaire de 2020 a provoqué une hausse temporaire du chômage, mais les réformes économiques ont permis de ramener le taux à 7,8 % en 2023.",
    "Selon les données de l'INSEE, le taux de chômage en France a fluctué entre 10 % et 7,8 % au cours des dix dernières années. Après une hausse notable en 2015, des mesures gouvernementales ont contribué à une baisse progressive. Cependant, la pandémie de COVID-19 en 2020 a inversé cette tendance temporairement avant de redescendre en 2023.",
    "Entre 2014 et 2023, le taux de chômage en France a varié considérablement. Après avoir atteint un sommet de 10,5 % en 2015, il a progressivement diminué, atteignant un minimum de 7,8 % en 2023. La pandémie de COVID-19 a temporairement perturbé cette tendance, augmentant le taux de chômage en 2020 et 2021.",
    "Au cours des dix dernières années, le taux de chômage en France a montré une tendance à la baisse. Après avoir atteint un pic en 2015, il a progressivement diminué, bien que la crise sanitaire de 2020 ait causé une augmentation temporaire. En 2023, le taux de chômage était de 7,8 %.",
    "L'évolution du taux de chômage en France de 2014 à 2023 révèle une baisse progressive après un pic en 2015. Bien que la pandémie ait provoqué une hausse temporaire, le taux de chômage a repris sa tendance à la baisse pour atteindre 7,8 % en 2023, selon l'INSEE.",
    "La cuisine française est mondialement reconnue pour sa diversité et son raffinement. Des plats emblématiques comme le coq au vin, la bouillabaisse et le bœuf bourguignon illustrent la richesse gastronomique du pays. Les vins et fromages français sont également très appréciés à l'international.",
    "Le système éducatif français est structuré en plusieurs niveaux, allant de l'école maternelle à l'université. L'éducation est obligatoire de 3 à 16 ans. Les élèves passent des examens nationaux comme le brevet des collèges et le baccalauréat, qui sont des étapes clés dans leur parcours scolaire.",
    "La France est l'un des leaders mondiaux dans la production d'énergie nucléaire. Environ 70 % de l'électricité du pays provient de centrales nucléaires. Cette dépendance permet à la France d'avoir une empreinte carbone relativement faible par rapport à d'autres pays européens.",
    "Le football est le sport le plus populaire en France, avec des millions de licenciés et de nombreux clubs répartis sur tout le territoire. L'équipe nationale, les Bleus, a remporté la Coupe du Monde de la FIFA en 1998 et 2018, ce qui a renforcé l'engouement pour ce sport parmi les Français.",
    "La France est riche en patrimoine culturel, avec des monuments emblématiques tels que la Tour Eiffel, le Mont Saint-Michel et le Château de Versailles. Le pays abrite également de nombreux musées de renommée mondiale, dont le Louvre et le Musée d'Orsay, qui attirent des millions de visiteurs chaque année."
]

dict_doc_rank = {doc : r for r, doc in enumerate(documents)}

for ass_func in [RG_YN, RG_S, RG_3L, RG_4L]:
    print("------------------------")
    reranked_documents = llm_reranking(
        query=question, 
        retrieved_documents=documents, 
        assessing_method=ass_func, 
        aggregating_method=expected_relevance_values
        )
    for i in range(5):
        print(f"Doc {i} th", reranked_documents[i])
        

#### Experiment LLM reranker 